# System Dependencies

To work with **Unstructured.io** and process documents effectively, a few system-wide dependencies are required. These tools handle PDFs, images, and file type detection.

---

## Poppler (poppler-utils) 🖥️

**Description:**  
Poppler is a **collection of command-line tools** for PDF processing. It can extract **text**, **images**, and **metadata** from PDFs.  

**Use in Project:**  
Unstructured uses Poppler to parse PDF documents and convert them into processable text or extract embedded images.  

**Common CLI Tools:**  
| Tool | Function |
|------|---------|
| `pdftotext` | Extracts selectable text from PDFs |
| `pdfimages` | Extracts embedded images from PDFs |
| `pdftoppm` / `pdftocairo` | Converts PDF pages to images (useful for OCR) |

---

## Tesseract (tesseract-ocr) 🔍

**Description:**  
Tesseract is an **Optical Character Recognition (OCR) engine**. It can read text from **scanned documents**, **images containing text**, or **image-based PDFs** and convert it into machine-readable text.  

**Use in Project:**  
- Extract text from scanned PDFs or image-heavy documents  
- OCR diagrams or images with embedded text  
- Works alongside tools like pdf2image or Poppler for converting pages to images before OCR  

---

## libmagic / python-magic-bin 🛠️

**Description:**  
libmagic is a **file type detection library**. It identifies the actual type of a file (PDF, Word, image, etc.) by analyzing its content rather than just relying on the file extension.  

**Use in Project:**  
- Helps Unstructured automatically determine the correct processing method for each document  
- On Windows, use **python-magic-bin**, a Python-friendly wrapper around libmagic  

---

✅ **Summary:**  
For your project using Unstructured.io:  
1. **Poppler** → PDF text, images, and metadata extraction  
2. **Tesseract OCR** → Text extraction from scanned pages and images  
3. **libmagic** → File type detection to route documents to the appropriate processing method  


In [ ]:
# for linux
# !apt-get install poppler-utils tesseract-ocr libmagic-dev

# for mac
# !brew install poppler tesseract libmagic

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
!pip install unstructured-client

### BY API CALL

In [1]:
import json
from typing import List

# Unstructured for document parsing
from unstructured.partition.pdf import partition_pdf
from unstructured.chunking.title import chunk_by_title

# LangChain components
from langchain_core.documents import Document
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
UNSTRUCTURED_API_KEY = "3HiBFzIghWShFPNY2q5JRzkKsabnCS"

In [3]:
import asyncio
import os
import json
import unstructured_client
from unstructured_client.models import shared, errors

client = unstructured_client.UnstructuredClient(
    api_key_auth=UNSTRUCTURED_API_KEY
)

async def partition_file_via_api(filename):
    req = {
        "partition_parameters": {
            "files": {
                "content": open(filename, "rb"),
                "file_name": os.path.basename(filename),
            },
            "strategy": shared.Strategy.AUTO,
            "vlm_model": "gpt-4o",
            "vlm_model_provider": "openai",
            "languages": ['eng'],
            "split_pdf_page": True,
            "split_pdf_allow_failed": True,
            "split_pdf_concurrency_level": 15
        }
    }

    try:
        res = await client.general.partition_async(request=req)
        return res.elements
    except errors.UnstructuredClientError as e:
        print(f"Error partitioning {filename}: {e.message}")
        return []

async def process_file_and_save_result(input_filename, output_dir):
    elements = await partition_file_via_api(input_filename)

    if elements:
        results_name = f"{os.path.basename(input_filename)}.json"
        output_filename = os.path.join(output_dir, results_name)

        with open(output_filename, "w") as f:
            json.dump(elements, f, indent=4)

def load_filenames_in_directory(input_dir):
    filenames = []
    for root, _, files in os.walk(input_dir):
        for file in files:
            if not file.endswith('.json'):
                filenames.append(os.path.join(root, file))

    return filenames

async def process_files():
    input_dir = "./input/"
    output_dir = "./output/"

    filenames = load_filenames_in_directory(input_dir)

    os.makedirs(output_dir, exist_ok=True)

    tasks = []

    for filename in filenames:
        tasks.append(
            process_file_and_save_result(filename, output_dir)
        )

    await asyncio.gather(*tasks)

await process_files()

In [4]:
import asyncio
import os
import json
import unstructured_client
from unstructured_client.models import shared, errors

# Initialize the client
client = unstructured_client.UnstructuredClient(
    api_key_auth=UNSTRUCTURED_API_KEY
)

# Declare global variable
elements = []

async def extract_pdf_to_json(pdf_path: str, output_dir: str):
    """
    Extracts content from a PDF using Unstructured API
    and saves it as a JSON file in the output directory.
    """
    global elements  # <-- declare that we will modify the global variable

    # Prepare request
    req = {
        "partition_parameters": {
            "files": {
                "content": open(pdf_path, "rb"), # File path
                "file_name": os.path.basename(pdf_path), # File name
            },
            "strategy": shared.Strategy.AUTO, # Strategy for Auto use "shared.Strategy.AUTO" for High perfomance use "hi_res"
            "vlm_model": "gpt-4o", # LLM model
            "vlm_model_provider": "openai", # LLM provider
            "languages": ["eng"], # Language
            "split_pdf_page": True, # Split by page
            "split_pdf_allow_failed": True, # Allow failed splits
            "split_pdf_concurrency_level": 15 ,# Concurrency level
            "extract_image_block_to_payload": True # Extract images as base64
        }
    }

    # Try extracting content
    try:
        res = await client.general.partition_async(request=req)
        elements = res.elements  # <-- modify global variable
    except errors.UnstructuredClientError as e:
        print(f"Error partitioning {pdf_path}: {e.message}")
        elements = []  # <-- also modifies global

    # Save result
    if elements:
        os.makedirs(output_dir, exist_ok=True)
        output_filename = os.path.join(output_dir, f"{os.path.basename(pdf_path)}.json")
        with open(output_filename, "w", encoding="utf-8") as f:
            json.dump(elements, f, indent=4)
        print(f"✅ Saved JSON to: {output_filename}")
    else:
        print(f"⚠️ No elements extracted from {pdf_path}")


# Example usage
pdf_file = r"D:\MultiModulRag\docs\NIPS-2017-attention-is-all-you-need-Paper.pdf"
output_folder = r"D:\MultiModulRag\docs"

# Run the async function
await extract_pdf_to_json(pdf_file, output_folder)

# Now 'elements' is available globally
print(f"Number of elements extracted: {len(elements)}")


INFO: HTTP Request: GET https://api.unstructuredapp.io/general/docs "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"


✅ Saved JSON to: D:\MultiModulRag\docs\NIPS-2017-attention-is-all-you-need-Paper.pdf.json
Number of elements extracted: 170


In [5]:
# All types of different atomic elements we see from unstructured
unique_types = list({item['type'] for item in elements})

print(unique_types)

['UncategorizedText', 'PageNumber', 'Table', 'Footer', 'Title', 'Formula', 'Image', 'NarrativeText']


In [6]:
elements[36]

{'type': 'PageNumber',
 'element_id': 'e66197b013c34f8553b126ada6b033e4',
 'text': '2',
 'metadata': {'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 2,
  'filename': 'NIPS-2017-attention-is-all-you-need-Paper.pdf'}}

In [7]:
# Gather all images
# Check the attribute for each element

# Often the type property is called 'element_type'
# Collect all URLs from NarrativeText elements
# Collect all URLs from all elements
urls = []

for el in elements:
    # Safely get metadata dictionary
    metadata = el.get("metadata", {})
    # Get links list if it exists
    links = metadata.get("links", [])
    for link in links:
        url = link.get("url")
        if url:
            urls.append(url)

# Remove duplicates
urls = list(set(urls))

print(f"Found {len(urls)} URLs:")
print(urls)


# Use https://codebeautify.org/base64-to-image-converter to view the base64 text

Found 42 URLs:
['figure.1', 'cite.neural_gpu', 'subsubsection.3.2.2', 'cite.extendedngpu', 'cite.DBLP:journals/corr/BritzGLL17', 'cite.kingma2014adam', 'cite.sutskever14', 'cite.lin2017structured', 'cite.sukhbaatar2015', 'cite.luong2015effective', 'cite.paulus2017deep', 'cite.sennrich2015neural', 'cite.wu2016google', 'cite.DBLP:journals/corr/SzegedyVISW15', 'cite.srivastava2014dropout', 'cite.shazeer2017outrageously', 'cite.he2016deep', 'figure.2', 'Hfootnote.2', 'cite.Kuchaiev2017Factorization', 'cite.structuredAttentionNetworks', 'cite.decomposableAttnModel', 'table.1', 'subsection.3.2', 'cite.bahdanau2014neural', 'cite.layernorm2016', 'table.3', 'table.2', 'cite.press2016using', 'cite.DBLP:journals/corr/ZhouCWLX16', 'https://github.com/tensorflow/tensor2tensor', 'cite.gruEval14', 'cite.JonasFaceNet2017', 'cite.xception2016', 'cite.graves2013generating', 'cite.NalBytenet2017', 'cite.hochreiter1997', 'cite.jozefowicz2016exploring', 'cite.hochreiter2001gradient', 'Hfootnote.1', 'cite.c

In [8]:
# Gather all table
alltables = []

for el in elements:
    # Check if the element has a type and it equals "Table"
    if el.get("type") == "Table":
        alltables.append(el)

# Optional: print how many tables were found
print(f"✅ Found {len(alltables)} tables.")

# Optional: print a preview
for i, table in enumerate(alltables[:3], start=1):  # show first 3 tables
    print(f"\n--- Table {i} ---")
    print(table["text"])

# Use https://jsfiddle.net/ to view the table html 


✅ Found 3 tables.

--- Table 1 ---
Layer Type Complexity per Layer Sequential Maximum Path Length Operations Self-Attention O(n2 · d) O(1) O(1) Recurrent O(n · d2) O(n) O(n) Convolutional O(k · n · d2) O(1) O(logk(n)) Self-Attention (restricted) O(r · n · d) O(1) O(n/r)

--- Table 2 ---
Model BLEU EN-DE EN-FR Training Cost (FLOPs) EN-DE EN-FR ByteNet [15] 23.75 Deep-Att + PosUnk [32] 39.2 1.0 · 1020 GNMT + RL [31] 24.6 39.92 2.3 · 1019 1.4 · 1020 ConvS2S [8] 25.16 40.46 9.6 · 1018 1.5 · 1020 MoE [26] 26.03 40.56 2.0 · 1019 1.2 · 1020 Deep-Att + PosUnk Ensemble [32] 40.4 8.0 · 1020 GNMT + RL Ensemble [31] 26.30 41.16 1.8 · 1020 1.1 · 1021 ConvS2S Ensemble [8] 26.36 41.29 7.7 · 1019 1.2 · 1021 Transformer (base model) 27.3 38.1 3.3 · 1018 Transformer (big) 28.4 41.0 2.3 · 1019

--- Table 3 ---
N odel [ dy dy €ls train PPL BLEU params Pyrop steps (dev) (dev) 100 base 6 512 2048 64 64 0.1 0.1 100K 4.92 25.8 65 512 512 5.29 24.9 128 128 5.00 25.5 (A) 32 32 491 25.8 B o > 16 16 5.01 25.4 (B)

### Chunk local